<a href="https://colab.research.google.com/github/davidcpage/mctc/blob/master/notebooks/01_CTC_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# default_exp normalisation

# Normalisation

> Partition function calculations.

In [6]:
#export
import numpy as np
import cupy as cp
import torch
import torch.nn as nn
from mctc.utils import *

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


## Preliminaries

Generate a test example:

In [7]:
#export
def generate_test_example(T, N, n_state, dtype=torch.float):
    return torch.rand((T, N, n_state, n_state), device=device, dtype=dtype, requires_grad=True)

### 1. Basic pytorch

In [8]:
T = 4032//5
N = 128
n_state = 8
Ms = generate_test_example(T, N, n_state)

In [9]:
#export
import torch

def _rescale(M):
    #T, N, n_state, n_state = M.shape
    Z = M.sum((2, 3), keepdim=True) / M.size(3)
    logZ = torch.log(Z).sum(0).reshape(-1)
    return M / Z, logZ    

@torch.jit.script
def logZ_py(M, alpha_0):
    M, logZ = _rescale(M)
    T, N, n_state, _ = M.shape
    alpha = alpha_0.unsqueeze(2)
    for i, M_t in enumerate(M.unbind(0)):
        alpha = M_t.bmm(alpha)
        if i % 32 == (T - 1) % 32:
            z = alpha.sum(1, keepdim=True)
            alpha = alpha/z
            logZ += torch.log(z.squeeze())
    return logZ

In [ ]:
#report(benchmark_fwd_bwd((lambda M, alpha_0: logZ_fwd(M, alpha_0).mean()), M, alpha_0))

### 3. Cupy

In [55]:
%%writefile cuda/fused_bmv.cu
__device__ __forceinline__ FLOAT max2(FLOAT a, FLOAT a1) {
    return a > a1 ? a : a1; 
}

__device__ __forceinline__ FLOAT logsumexp2(FLOAT a, FLOAT a1) {
    FLOAT maxa = max2(a, a1); 
    return maxa + log(exp(a-maxa) + exp(a1-maxa));
}

__device__ __forceinline__ FLOAT add(FLOAT a, FLOAT b) {return a + b;}
__device__ __forceinline__ FLOAT mul(FLOAT a, FLOAT b) {return a * b;}

extern "C" __global__ void fwd(
    FLOAT* __restrict__ alpha,
    const FLOAT* __restrict__ Ms, 
    int T, int N, int n_state
) {
    // Ms is shape (T, N, n_state, n_state)
    // alpha is shape (T + 1, N, n_state)
    // assumes blockDim = (N, 1, 1) and threadDim = (n_state, 1, 1)

    int bx = blockIdx.x, tx = threadIdx.x;
    if (tx >= n_state) return;
    FLOAT u;
    for (int t = 0; t < T; t++) {
        int j = (t * N + bx) * n_state;
        u = MUL(Ms[(j + tx) * n_state], alpha[j]);
        for (int i = 1; i < n_state; i++) {
            u = SUM(u, MUL(Ms[(j + tx) * n_state + i], alpha[j + i]));
        }
        alpha[j + (N * n_state) + tx] = u;
        __syncthreads();
    }
  }

Overwriting mctc/cuda/fused_bmv.cu


In [57]:
#export
from mctc.ctc import Log

cupy_funcs = {
    (torch.float32, Log): load_cupy_func('cuda/fused_bmv.cu', 'fwd', FLOAT='float', MUL='add', SUM='logsumexp2'),
    (torch.float64, Log): load_cupy_func('cuda/fused_bmv.cu', 'fwd', FLOAT='double', MUL='add', SUM='logsumexp2'),
}

def fused_batch_Mv(Ms, alpha_0):
    T, N, n_state, _ = Ms.shape
    alpha = Ms.new_zeros((T + 1, N, n_state))
    alpha[0] = alpha_0
    with cp.cuda.Device(Ms.device.index):
        cupy_funcs[(Ms.dtype, Log)](
            grid=(N, 1, 1), 
            block=(n_state, 1, 1), 
            args=(alpha.data_ptr(), Ms.contiguous().data_ptr(), T, N, n_state)
        )
    return alpha

class FusedBatchMvLogspace(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Ms, alpha_0):
        alpha = fused_batch_Mv(Ms, alpha_0)
        ctx.save_for_backward(Ms, alpha)
        return alpha[-1]

    @staticmethod
    def backward(ctx, g):
        Ms, alpha = ctx.saved_tensors
        T, N, n_state, _ = Ms.shape
        beta = fused_batch_Mv(Ms.transpose(2,3).flip(0), g)
        Ms_grad = Ms + alpha[:-1,:,None,:] + beta[:-1,:,:,None].flip(0)
        Ms_grad = torch.softmax(Ms_grad.reshape(T, N, -1), 2).reshape(T, N, n_state, n_state)
        alpha_0_grad = beta[-1]
        return Ms_grad, alpha_0_grad

fused_batch_Mv_logspace = FusedBatchMvLogspace.apply